## 기상청 API 허브

In [ ]:
import sys

print("--sys.version—")
print(sys.version)

DATA 수집 목표 : 2012 ~ (최소기간)

https://apihub.kma.go.kr/

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import urllib
import urllib.request
from urllib.parse import unquote

import json

import requests
import pprint

from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

In [ ]:
import pickle

In [ ]:
with open('단기해상예보.pkl', 'wb') as f:
    pickle.dump(lst, f)

In [ ]:
with open('단기해상예보.pkl', 'rb') as f:
    lst = pickle.load(f)

In [ ]:
for i in lst:
    print(i[:5])

### Text

In [ ]:
year = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
year2 = [2012, 2016, 2020]#, 2023]
# month = [1, 4, 7, 10]

In [ ]:
def one_year_term(year):
    url = f'https://apihub.kma.go.kr/api/typ01/url/kma_snow2.php?tm={year+1}01010000&tm_st={year}01010000&snow=2&help=0&authKey=vBjTN-oHQjaY0zfqB_I2uw'
    response = requests.get(url)
    text = response.text.split('\n')
    return text
    

In [ ]:
def six_month_term(year, a, b):
    
    url = f'https://apihub.kma.go.kr/api/typ01/url/fct_afs_do.php?reg=&tmfc1={year}{a}0100&tmfc2={year}{b}0100&disp=1&help=0&authKey=yEtC0d2xQmKLQtHdsWJi5g'
    response = requests.get(url)
    text = response.text.split('\n')
    return text
    

In [ ]:

url = f'https://apihub.kma.go.kr/api/typ01/url/kma_snow2.php?tm=202303130000&tm_st=202301010000&snow=2&help=0&authKey=zxiqblJaSgSYqm5SWioEeA'
response = requests.get(url)
text = response.text.split('\n')


In [ ]:
text[-5:]

In [ ]:
lst = []

for i in tqdm(year):
    lst.append(one_year_term(i))

In [ ]:
len(lst)

In [ ]:
# 2012, 2016, 2020 : 기간 초과

In [ ]:
lst_half = []

for i in tqdm(year2):
    lst_half.append(short_term_half(i, '01', '06'))
    lst_half.append(short_term_half(i, '07', '12'))
    

In [ ]:
len(lst_half)

In [ ]:
lst_half[4][:-5]

In [ ]:
lst[0]

In [ ]:
for i in range(11):
    print(lst[i][:5])

In [ ]:
# columns 추출
col_name = text[1].split()
col_name = col_name[1:]
print(col_name)
print(len(col_name))

In [ ]:
# col_name = [i.strip() for i in col_name]

In [ ]:
# col_name[0] = 'TM_FC'

In [ ]:
col_name

In [ ]:
for i in lst:
    print(i[-3:])

In [ ]:
# df_2013, df_2014, df_2015, 
# df_2017, df_2018, df_2019, 
# df_2021, df_2022

In [ ]:
len(lst[0])

In [ ]:
lst[1][2].split(',')[1][:4]

In [ ]:
# 단기 예특보
# lst 내 text, Data Frame으로 만들기

def to_df(lst):
    for i in tqdm(range(len(lst))):
        if len(lst[i]) > 2:
            year = lst[i][2].split(',')[1][:4]
            globals()['df_{}'.format(year)] = pd.DataFrame(lst[i][2:-2])[0].str.split(',', expand=True)
#             'df_{}'.format(year).columns == col_name
#             return 'df_{}'.format(year)
#         else:
#             pass

In [ ]:
to_df(lst_half)

In [ ]:
lst[1]

In [ ]:
lst[1][3].split()[0][:4]

In [ ]:
# 적설량
# lst 내 text, Data Frame으로 만들기

def to_df2(lst):
    for i in tqdm(range(len(lst))):
        year = lst[i][3].split()[0][:4]
        globals()['df_{}_snow'.format(year)] = pd.DataFrame(lst[i][3:-2])[0].str.split(expand=True)


In [ ]:
to_df2(lst)

In [ ]:
df_2022_snow

In [ ]:
text

In [ ]:
df = pd.DataFrame(text[3:-2])[0].str.split(expand=True)
# df3.drop(columns = [19], inplace=True)
df.columns = col_name
df

In [ ]:
df1

In [ ]:
df_2020

In [ ]:
df[9].value_counts()

In [ ]:
'''
# 띄어쓰기에 따라 나눠진 컬럼 합치기

con_cols = [18, 19, 20, 21]

df['con_cols'] = df[con_cols].apply(lambda row : ' '.join(row.values.astype(str)), axis=1)
df.drop(columns = con_cols, inplace = True)
'''

In [ ]:
df = df.iloc[:,:11]

In [ ]:
df.drop(columns = 11, inplace=True)

In [ ]:
df_2012_snow

In [ ]:
# df_2013, df_2014, df_2015, df_2017, df_2018, df_2019, df_2021, df_2022

df_snow = pd.concat([df_2012_snow, df_2013_snow, df_2014_snow, df_2015_snow, df_2016_snow, df_2017_snow, df_2018_snow, df_2019_snow, df_2020_snow, df_2021_snow, df_2022_snow])
# df1.drop(columns = [19], inplace=True)
df_snow.columns = col_name
df_snow

In [ ]:
df_snow = df_snow.sort_values(by='YYMMDDHHMI')
df_snow = df_snow.reset_index(drop=True)
df_snow

In [ ]:
df_snow = df_snow.drop_duplicates()
df_snow = df_snow.reset_index(drop=True)
df_snow

In [ ]:
df_snow.to_csv('적설관측(~2022).csv')

In [ ]:
df2 = pd.concat([df_2012, df_2016, df_2020])
df2.drop(columns = [19], inplace=True)
df2.columns = col_name
df2

In [ ]:
df = pd.concat([df1, df2, df3])
df.sort_values(by='TM_FC', inplace=True)
df = df.reset_index(drop=True)
df

In [ ]:
df[:20]

In [ ]:
df = df.drop_duplicates()
df

In [ ]:
df['TM_FC'].value_counts()

In [ ]:
# 필요없는 COLUMN 제거 구역코드(REG_ID), 예보관ID(MAN_ID), 예보관명(MAN_FC)
# 구간(MOD) 12, 24 시간
# SKY + PREP 한글화 = WF

df.drop(columns = ['REG_ID', 'MAN_ID', 'MAN_FC'], inplace=True)

In [ ]:
df.to_csv('단기해상예보.csv')

### XML

In [ ]:
url = 'https://apihub.kma.go.kr/api/typ02/openApi/SfcYearlyInfoService/getTyphoonList?pageNo=1&numOfRows=10&dataType=XML&year=2022&authKey=_v7wW5IUQqu-8FuSFNKrLA'

response = requests.get(url)

# xml 내용
content = response.text

# 깔끔한 출력 위한 코드
pp = pprint.PrettyPrinter(indent=4)
print(pp.pprint(content))

In [ ]:
from os import name
import xml.etree.ElementTree as et
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

#bs4 사용하여 item 태그 분리
xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('info') # item 으로 구분되어 있는 경우도 있음
print(rows)

In [ ]:
# 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

In [ ]:
#xml값 DataFrame으로 만들기
xml_df = pd.DataFrame(row_list, columns=name_list)
print(xml_df.head())

In [ ]:
xml_df

In [ ]:
xml_df.to_csv('우리나라영향태풍_2021.csv')

# Data

In [ ]:
# df1 = pd.read_csv('H:/내 드라이브/취업/Contest/2307_PublicData/data/단기육상예보_1차.csv', index_col=0)
# df2 = pd.read_csv('H:/내 드라이브/취업/Contest/2307_PublicData/data/단기육상예보_2차.csv', index_col=0)
# df3 = pd.read_csv('H:/내 드라이브/취업/Contest/2307_PublicData/data/단기육상예보_3차.csv', index_col=0)

df = pd.read_csv('H:/내 드라이브/취업/Contest/2307_PublicData/data/단기육상예보.csv', index_col=0)

In [ ]:
df1

In [ ]:
df2

In [ ]:
df3

In [ ]:
df3['TM_FC'] = df3['TM_FC'].astype('int64')

In [ ]:
df = pd.concat([df1, df2, df3]).reset_index()
df.drop(columns='index', inplace=True)
df

In [ ]:
df = df.sort_values(by='TM_FC', ascending=True)
df = df.reset_index(drop=True)
df

In [ ]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df

------

In [15]:
df = pd.read_csv('./적설관측(~2022).csv', index_col = 0)
df1 = pd.read_csv('./2023.csv', index_col = 0)

In [ ]:
df1.reset_index(inplace=True, drop=True)
df1

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.reset_index(inplace=True, drop=True)
df

In [19]:
dd = pd.concat([df, df1])#.reset_index(drop=True)
# df3 = df3.reset_index(drop=True)
dd

,YYMMDDHHMI,STN,STN.1,LON,LAT,STN.2,SD_TOT,SD_DAY,SD_HR3,SD_D01,SD_H03,SD_H24,SD_H01
0,201201010000,90,속초,128.564720,38.250850,000-----,1.6,-999.0,-999.0,0.0,0.0,0.0,0.0
1,201201010000,172,고창,126.599000,35.348950,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,201201010000,283,경주시(공),129.200909,35.817461,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,201201010000,102,백령도,124.630460,37.966110,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,201201010000,98,동두천,127.060690,37.901860,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
787705,202303122000,5343,포항,129.379630,36.032590,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
787706,202303122100,5343,포항,129.379630,36.032590,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
787707,202303122200,5343,포항,129.379630,36.032590,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
787708,202303122300,5343,포항,129.379630,36.032590,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [20]:
dd = dd.reset_index(drop=True)
dd

,YYMMDDHHMI,STN,STN.1,LON,LAT,STN.2,SD_TOT,SD_DAY,SD_HR3,SD_D01,SD_H03,SD_H24,SD_H01
0,201201010000,90,속초,128.564720,38.250850,000-----,1.6,-999.0,-999.0,0.0,0.0,0.0,0.0
1,201201010000,172,고창,126.599000,35.348950,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,201201010000,283,경주시(공),129.200909,35.817461,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,201201010000,102,백령도,124.630460,37.966110,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,201201010000,98,동두천,127.060690,37.901860,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7501929,202303122000,5343,포항,129.379630,36.032590,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
7501930,202303122100,5343,포항,129.379630,36.032590,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
7501931,202303122200,5343,포항,129.379630,36.032590,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
7501932,202303122300,5343,포항,129.379630,36.032590,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [18]:
type(dd)

NoneType

In [13]:
df1

,YYMMDDHHMI,STN,STN.1,LON,LAT,STN.2,SD_TOT,SD_DAY,SD_HR3,SD_D01,SD_H03,SD_H24,SD_H01
0,202301010000,90,속초,128.56473,38.25085,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
1,202301010100,90,속초,128.56473,38.25085,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,202301010200,90,속초,128.56473,38.25085,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,202301010300,90,속초,128.56473,38.25085,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,202301010400,90,속초,128.56473,38.25085,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
787705,202303122000,5343,포항,129.37963,36.03259,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
787706,202303122100,5343,포항,129.37963,36.03259,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
787707,202303122200,5343,포항,129.37963,36.03259,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
787708,202303122300,5343,포항,129.37963,36.03259,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [ ]:
for i in df2['STN.1'].value_counts().index : print(i)

In [21]:
dd.to_csv('적설관측자료.csv')